<a href="https://colab.research.google.com/github/tbradshaw91/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/Instacart_Boozy_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing all the necessary packages

In [0]:
!pip install seaborn --upgrade
import os
os.kill(os.getpid(), 9)

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [3]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [4]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


### Importing Products and Orders 


In [0]:
products = pd.read_csv('products.csv')

order_products = pd.concat([pd.read_csv('order_products__prior.csv'), # Another interesting way to read in multiple pandas functions
                            pd.read_csv('order_products__train.csv')]) # "composing functions"

orders = pd.read_csv('orders.csv')

**Looking at Product Names to Find Wine**

In [6]:
products['product_name'].head()
products.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')

In [7]:
pd.options.display.max_columns = 500
aisles = pd.read_csv('aisles.csv')
aisles.columns.tolist()

['aisle_id', 'aisle']

In [0]:
aisles['aisle'].tail(50) #red wines #white wines #specialty wines champagnes

In [9]:
departments = pd.read_csv('departments.csv')
departments.columns.tolist()
#departments['department'] # I want the alcohol one(4) #snacks is secondary (18)

['department_id', 'department']

In [0]:
# Merged products, aisle, and departments
merged = (products[['aisle_id', 'product_name', 'department_id', 'product_id']]
          .merge(aisles[['aisle_id', 'aisle']])
          .merge(departments[['department_id', 'department']]))

In [11]:
merged.shape, products.shape, aisles.shape, departments.shape

((49688, 6), (49688, 4), (134, 2), (21, 2))

In [12]:
# It worked
merged.head()

,aisle_id,product_name,department_id,product_id,aisle,department
0,61,Chocolate Sandwich Cookies,19,1,cookies cakes,snacks
1,61,Nutter Butter Cookie Bites Go-Pak,19,78,cookies cakes,snacks
2,61,Danish Butter Cookies,19,102,cookies cakes,snacks
3,61,Gluten Free All Natural Chocolate Chip Cookies,19,172,cookies cakes,snacks
4,61,Mini Nilla Wafers Munch Pack,19,285,cookies cakes,snacks


### **Aisle Conditions**

In [0]:
# What condion will filter merged DF to just the 2 products that we care about?
# OMFG this is exactly what I was attempting yesterday


condition1 = ((merged['aisle'] == 'red wines') | 
              (merged['aisle'] == 'white wines') |
              (merged['aisle'] == 'specialty wines champagnes')|
              (merged['aisle'] == 'spirits'))

# to this....
drunk_aisles = ['red wines', 'white wines','specialty wines champagnes', 'spirits' ]

condition1 = merged['aisle'].isin(drunk_aisles)
subset_aisles = merged[condition1]

In [14]:
subset_aisles.shape

(669, 6)

In [15]:
subset_aisles.head()

,aisle_id,product_name,department_id,product_id,aisle,department
46532,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol
46533,134,Brut Rosé,5,150,specialty wines champagnes,alcohol
46534,134,Champagne,5,1808,specialty wines champagnes,alcohol
46535,134,Brut Cuvee Sparkling Champagne,5,2022,specialty wines champagnes,alcohol
46536,134,California Champagne,5,2216,specialty wines champagnes,alcohol


### Departments Conditions

In [0]:
condition2 = ((merged['department'] == 'alcohol'))

# to this....
dds = ['alcohol']

condition2 = merged['department'].isin(dds)
subset_dds = merged[condition2]

In [17]:
subset_dds.head()

,aisle_id,product_name,department_id,product_id,aisle,department
46532,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol
46533,134,Brut Rosé,5,150,specialty wines champagnes,alcohol
46534,134,Champagne,5,1808,specialty wines champagnes,alcohol
46535,134,Brut Cuvee Sparkling Champagne,5,2022,specialty wines champagnes,alcohol
46536,134,California Champagne,5,2216,specialty wines champagnes,alcohol


In [18]:
subset_dds.shape

(1054, 6)

In [0]:
grand_merge = pd.merge(subset_aisles, subset_dds)

In [20]:
grand_merge.shape

(669, 6)

## Merging Many to One

In [21]:
orders.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')

In [22]:
order_products.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')

In [0]:
le_orders = pd.merge(orders, order_products)

In [24]:
le_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [0]:
final = pd.merge(grand_merge, le_orders)

In [26]:
final.head()

,aisle_id,product_name,department_id,product_id,aisle,department,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
0,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,1114232,626,prior,43,1,12,4.0,3,0
1,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,2390070,21631,prior,3,5,12,6.0,9,0
2,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,2375085,52689,prior,2,6,9,2.0,3,0
3,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,3384749,52689,prior,3,2,21,3.0,5,1
4,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,1091852,52689,prior,6,6,8,2.0,1,1


In [32]:
final['product_name'].value_counts().head()

Sauvignon Blanc       8541
Cabernet Sauvignon    6352
Chardonnay            6346
Vodka                 5666
Pinot Noir            4116
Name: product_name, dtype: int64

In [33]:
final.isnull().sum()

aisle_id                     0
product_name                 0
department_id                0
product_id                   0
aisle                        0
department                   0
order_id                     0
user_id                      0
eval_set                     0
order_number                 0
order_dow                    0
order_hour_of_day            0
days_since_prior_order    8742
add_to_cart_order            0
reordered                    0
dtype: int64

In [0]:
# Getting rid of NaNs
final['days_since_prior_order'] = final['days_since_prior_order'].replace(np.nan, 0)

In [0]:
# Getting rid of eval_set
final.drop('eval_set', axis=1)

In [45]:
final.head()

,aisle_id,product_name,department_id,product_id,aisle,department,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
0,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,1114232,626,43,1,12,4.0,3,0
1,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,2390070,21631,3,5,12,6.0,9,0
2,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,2375085,52689,2,6,9,2.0,3,0
3,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,3384749,52689,3,2,21,3.0,5,1
4,134,Mirabelle Brut Rose,5,52,specialty wines champagnes,alcohol,1091852,52689,6,6,8,2.0,1,1


## Plotting

**I would like to look at:

*   Top Alcohols Ordered
*    Alcohol vs DOW
*    Alcohol vs HOD


In [0]:
sns.catplot(x='aisle', y='order_dow', col='product_name',
           kind='bar', data=final, color='hotpink');

In [0]:
final.pivot_table(index='year', columns='product_name', values='passengers')